In [ ]:
import torch
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.model_selection import KFold
from scipy.signal import savgol_filter

import itertools
import sys
import os
sys.path.append(os.path.abspath('../src'))

from helper import load_data, data_source_release
from helper import preprocess

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib.style as style
style.use('seaborn-poster') #sets the size of the charts
style.use('ggplot')

plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 1
plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 1
plt.rcParams['grid.color'] = "#cccccc"

In [ ]:
def loadModel(model_type, model_name):
    return torch.load('../export/{}/{}.pt'.format(model_type, model_name))

def getScenarioData(scenario):
    sc_dict = data_source_release.get_sc_dict()
    data_dir = data_source_release.get_data_dir()
    runtimes, features, _ = load_data.get_data(scenario=scenario, data_dir=data_dir, sc_dict=sc_dict, retrieve=sc_dict[scenario]['use'])
    return np.array(runtimes), np.array(features)

In [ ]:
# Load data and preprocess
scenario = 'clasp_factoring'
runtimes, features = getScenarioData(scenario)
fold = 9
idx = list(range(runtimes.shape[0]))

kf = KFold(n_splits=10, shuffle=True, random_state=0)
train_idx, validate_idx = list(kf.split(idx))[fold]
X_trn_flat, X_vld_flat, _, _, _, _ = preprocess.preprocess(features, runtimes, train_idx, validate_idx, 100, 0, 0)

In [ ]:
device = torch.device("cpu")
features = torch.tensor(X_trn_flat, dtype=torch.float).to(device)
data_row = 0
X = np.linspace(-8, 8, 600)

In [ ]:
# Load Bayes DistNet, sample distribution
model_bayes = loadModel('bayes_distnet', 'bayes_distnet_clasp_factoring_BAYESIAN_LOGNORMAL_16_0_6_0')
model_bayes.to(device)
model_bayes.eval()

mean_bayes = []
std_bayes = []

# Iterate through values of feature X-20
for x in X:
    F = features[data_row].repeat(16,1).to(device) + x
    # Batch (32) * 16 = 512 samples
    temp = []
    for _ in range(32):
        net_out, _ = model_bayes(F)
        temp += net_out.flatten().tolist()
        
    mean_bayes.append(np.mean(temp))
    std_bayes.append(np.std(temp) * 0.6745)

In [ ]:
# Load DistNet, get parameters of distribution
model_distnet = loadModel('distnet', 'distnet_{}_LOGNORMAL_16_0_6_0'.format(scenario))
model_distnet.to(device)
model_distnet.eval()

means_distnet = []
stds_distnet = []

# Iterate through values of feature X-20
for x in X:
    F = features[data_row].repeat(1,1).to(device) + x
    net_out = model_distnet(F)[0].tolist()
    dist = stats.lognorm(s=net_out[0], loc=0, scale=net_out[1])
    means_distnet.append(dist.mean())
    stds_distnet.append(dist.std() * 0.6745)

In [ ]:
Y_bayes = savgol_filter(mean_bayes, 11, 3)
LB_bayes = savgol_filter([y - ci for y, ci in zip(mean_bayes, std_bayes)], 11, 3)
UB_bayes = savgol_filter([y + ci for y, ci in zip(mean_bayes, std_bayes)], 11, 3)

Y_distnet = savgol_filter(means_distnet, 11, 3)
LB_distnet = savgol_filter([y - ci for y, ci in zip(means_distnet, stds_distnet)], 11, 3)
UB_distnet = savgol_filter([y + ci for y, ci in zip(means_distnet, stds_distnet)], 11, 3)

fig, ax = plt.subplots()

# Bayes
ax.plot(X, Y_bayes, color='#348ABD', label="Bayes DistNet")
ax.fill_between(X, LB_bayes, UB_bayes, color='#348ABD', alpha=.15)
# Distnet
ax.plot(X, Y_distnet, color='#E24A33', label="DistNet")
ax.fill_between(X, LB_distnet, UB_distnet, color='#E24A33', alpha=.15)

# Chart attributes
handles, labels = ax.get_legend_handles_labels()

# Tick/labels
ax.tick_params(axis='both', labelsize=22)
ax.set_ylabel('Distribution')  
ax.set_xlabel('Input Offset')  
ax.xaxis.label.set_size(24)
ax.yaxis.label.set_size(24)

lgd = ax.legend(handles, labels, loc='upper left', prop={'size': 20})
lgd.get_frame().set_edgecolor('black')
plt.autoscale(enable=True, axis='x', tight=True)

# plt.savefig('../export/images/predictive_uncertainty.svg', format='svg', dpi=1200, bbox_inches='tight')